## Preliminaries

Setting-up

In [1]:
setwd(base::dirname(getwd()))
getwd()

[1] "J:/library/helminthiases/preliminary"

<br>

Libraries

In [2]:
.activate <- function (x){
    library(x, character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE, verbose = FALSE)
    if (x == 'rmarkdown') {library(tinytex)}
}
log <- lapply(X = c('magrittr', 'dplyr', 'tibble', 'ggplot2', 'stringr', 'lubridate'), .activate)

Warning message:
"package 'magrittr' was built under R version 4.1.3"
Warning message:
"package 'dplyr' was built under R version 4.1.3"
Warning message:
"package 'tibble' was built under R version 4.1.3"
Warning message:
"package 'ggplot2' was built under R version 4.1.3"
Warning message:
"package 'stringr' was built under R version 4.1.3"
Warning message:
"package 'lubridate' was built under R version 4.1.3"


<br>

## Data Sources

In [3]:
files <- list.files(path = file.path(getwd(), 'warehouse', 'missing', 'disaggregates'),
                    full.names = TRUE)

<br>
<br>

## Null Correlation

In [4]:
source(file = 'R/missing/Steps.R')

In [5]:
cores <- parallel::detectCores() - 2
doParallel::registerDoParallel(cores = cores)
clusters <- parallel::makeCluster(cores)
X <- parallel::clusterMap(clusters, fun = Steps, files, USE.NAMES = FALSE)
parallel::stopCluster(clusters)
rm(clusters, cores)

In [6]:
correlations <- dplyr::bind_rows(X)

<br>

### Preserve

Storage

In [7]:
storage <- file.path(getwd(), 'warehouse', 'missing', 'correlation', 'cramer.csv')
if (file.exists(storage)) {
  base::unlink(x = storage, recursive = TRUE)
}
dir.create(path = base::dirname(storage), recursive = TRUE)

<br>

Write

In [8]:
utils::write.table(x = correlations,
                   file = storage,
                   append = FALSE,
                   sep = ',',
                   row.names = FALSE,
                   col.names = TRUE,
                   fileEncoding = 'UTF-8')